<a href="https://colab.research.google.com/github/Biswajitnahak2003/GenAIByLangchain/blob/main/langchain_tools/tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Small Project: Weather Information**(using tools binding, calling and execution)


1. Tool Binding

This is the first step where we make the LLM aware of the tools it can use. We use the .bind_tools() method to attach a list of tools to the model. *

 2. Tool Calling

This is the model's response when it decides it needs to use a tool to answer the query. *

 3. Tool Execution

This is where, the programmer, take the model's instruction and actually run the corresponding Python function. *

In [1]:
!pip install langchain-google-genai

In [2]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_core.messages import HumanMessage
import json


In [4]:
@tool
def getweather(city: str) -> str:
    """Find the weather of the city given and return its weather info"""
    weather_data = {
        "bhubaneswar": "32°C and sunny",
        "mumbai": "28°C and humid with a chance of rain",
        "delhi": "35°C and hazy",
        "gothapatna": "31°C with clear skies"
    }
    return weather_data.get(city.lower(), "Weather information not available")


In [5]:

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature = 2
)
tools = [getweather]
llm_with_tools = llm.bind_tools(tools)
query = "what's the weather in gothapatna?"
ai_message = llm_with_tools.invoke(query)

print(ai_message.tool_calls)

[{'name': 'getweather', 'args': {'city': 'gothapatna'}, 'id': '1d4e36cb-07b5-40ee-ae66-93dd68acecbd', 'type': 'tool_call'}]


In [6]:

for tool_call in ai_message.tool_calls:
    tool_name_from_model = tool_call["name"]
    selected_tool = None
    for tool in tools:
        if tool.name == tool_name_from_model:
            selected_tool = tool
            break

    if selected_tool:
        tool_output = selected_tool.invoke(tool_call["args"])
        print(f"Tool Output: {tool_output}")

        conversation_history = [
            HumanMessage(content=query),
            ai_message,
            ToolMessage(tool_output, tool_call_id=tool_call["id"])
        ]

        final_response = llm_with_tools.invoke(conversation_history)

        print("\n--- Final AI Response ---")
        print(final_response.content)
    else:
        print(f"Error: Could not find a tool named '{tool_name_from_model}'.")

Tool Output: 31°C with clear skies

--- Final AI Response ---
The weather in Gothapatna is 31°C with clear skies.
